<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ricardo Torres
- Nombre de alumno 2: Simon Estrada


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/RicardoTorresP/Laboratorio-9`

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_09/Lab_09'
except:
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv(path+'/sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import FunctionTransformer

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(df.drop('quantity', axis=1), df['quantity'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

In [ ]:
def extract_date_info(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)
date_transformer = FunctionTransformer(extract_date_info, validate=False)

¿Cómo se interpreta esta métrica para el contexto del negocio?

Respuesta:

El MAE, o Error Absoluto Medio, es una métrica fundamental en el contexto empresarial que cuantifica la discrepancia promedio entre las predicciones de un modelo y los valores reales. En términos sencillos, representa cuánto se desvían, en promedio, las estimaciones del modelo de las cifras reales.

Un MAE más bajo es indicativo de un rendimiento más preciso del modelo, ya que implica que las predicciones tienen una desviación menor con respecto a los datos reales. Por lo tanto, al minimizar el MAE, se busca optimizar la capacidad del modelo para realizar predicciones precisas. Esto es crucial en entornos empresariales donde las estimaciones precisas son esenciales para la toma de decisiones estratégicas, como la gestión de inventarios, la planificación de la producción o la optimización de campañas de marketing.

In [ ]:
numeric_features = ['lat', 'long', 'pop', 'price']
categorical_features = [ 'city', 'shop', 'brand', 'container','day','month','year','capacity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [ ]:
dummy_model = DummyRegressor(strategy='mean')
xgb_model = XGBRegressor()

pipeline_dummy2 = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', dummy_model)
])

pipeline_xgb2 = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

In [ ]:
# Fit and evaluate the DummyRegressor pipeline
pipeline_dummy2.fit(X_train, y_train)
y_val_pred_dummy2 = pipeline_dummy2.predict(X_val)
mae_dummy2 = mean_absolute_error(y_val, y_val_pred_dummy2)
print(f'MAE for DummyRegressor: {mae_dummy2}')

joblib.dump(pipeline_dummy2, 'dummy_model.pkl')


# Fit and evaluate the XGBRegressor pipeline
pipeline_xgb2.fit(X_train, y_train)
y_val_pred_xgb2 = pipeline_xgb2.predict(X_val)
mae_xgb2 = mean_absolute_error(y_val, y_val_pred_xgb2)
print(f'MAE for XGBRegressor: {mae_xgb2}')

joblib.dump(pipeline_xgb2, 'xgb_model.pkl')

MAE for DummyRegressor: 13308.134750658153
MAE for XGBRegressor: 2406.0000457967394


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el "DummyRegressor"?


Al implementar este algoritmo se tiene una disminución significativa en el Valor Absoluto Medio (MAE), esto sugiere una mejora sustancial en la precisión del modelo. El MAE es una métrica que cuantifica la magnitud promedio de los errores entre las predicciones del modelo y los valores reales. En este caso, el MAE ha disminuido de 13308 a 2406 al pasar del DummyRegressor al XGBRegressor.

Esta reducción en el MAE indica que el XGBRegressor está proporcionando predicciones más precisas en comparación con el DummyRegressor. Un MAE más bajo implica que las predicciones del modelo están más cerca de los valores reales, lo que se traduce en un rendimiento mejorado.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [ ]:
class SparseToDataFrameTransformer:
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = pd.DataFrame(X.toarray(), columns=[f'col_{i}' for i in range(X.shape[1])])

        df.columns = numeric_features + list(df.columns[4:])

        return df

In [ ]:
xgb_model_with_constraints = XGBRegressor(monotone_constraints={'price': -1})

pipeline_xgb_constraints = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('to_dataframe', SparseToDataFrameTransformer()),
    ('regressor', xgb_model_with_constraints)
])

pipeline_xgb_constraints.fit(X_train, y_train)
y_val_pred_xgb2 = pipeline_xgb_constraints.predict(X_val)
mae_xgb_constraints = mean_absolute_error(y_val, y_val_pred_xgb2)
print(f'MAE for XGBRegressor: {mae_xgb_constraints}')
joblib.dump(pipeline_xgb_monotone, 'xgb_model_monotone.pkl')

MAE for XGBRegressor: 2327.1220129443423


¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

**Respuesta:**


Al incorporar la relación monotona entre el precio y la cantidad, observamos una notable mejora en la precisión del modelo. El error medio absoluto (MAE) disminuye de 2406 a 2327, lo que sugiere que la inclusión de esta relación mejora la capacidad del modelo para predecir las demandas. Este cambio en el error respalda la validez de la relación identificada por el modelo entre el precio y la cantidad demandada.

Por lo tanto, la conclusión es que sí, la inclusión de la relación propuesta por su amigo parece ser acertada.



## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

---
Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?

**Respuesta:**

Los hiperparámetros y sus roles son los siguientes:

learning_rate: Define la tasa de aprendizaje del modelo. \
n_estimators: Número de árboles (estimadores) en el ensamble. \
max_depth: Profundidad máxima de cada árbol. \
max_leaves: Número máximo de hojas por árbol. \
min_child_weight: Mínimo peso requerido para crear un nuevo nodo en el árbol. \
reg_alpha: Término de regularización L1. \
reg_lambda: Término de regularización L2. \

En cuanto a los rangos se tiene lo siguiente:



*   **learning_rate**: El rango (0.001, 0.1) es común para este hiperparámetro en modelos de gradient boosting.
*   **n_estimators**: El rango (50, 1000) para el número de estimadores (o árboles) en el bosque parece razonable. Es importante equilibrar el desempeño y la eficiencia computacional.

* max_depth: Un rango de (3, 10) para la profundidad máxima de los árboles es razonable. Esto evita árboles demasiado profundos que podrían llevar a sobreajuste.

* **max_leaves**: No es común tener un hiperparámetro específico para el número máximo de hojas.

* **min_child_weight**: El rango (1, 5) es razonable para este hiperparámetro, que controla la cantidad mínima de instancias necesarias en cada hoja.

* **reg_alpha y reg_lambda**: Los rangos (0, 1) para ambos parámetros de regularización son apropiados. Controlan la regularización L1 y L2, respectivamente. Estos valores más altos penalizan los coeficientes más grandes y pueden ayudar a prevenir el sobreajuste.



In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
import joblib
import time

# Fijar la semilla para reproducibilidad
seed = 42
import numpy as np
np.random.seed(seed)

# Definir los rangos de optimización
param_ranges = {
    'learning_rate': (0.001, 0.1),
    'n_estimators': (50, 1000),
    'max_depth': (3, 10),
    'max_leaves': (0, 100),
    'min_child_weight': (1, 5),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1),
}

encoder_ranges = {
    'min_frequency': (0.0, 1.0)
}


In [ ]:
def objective(trial):
    # Definir el espacio de búsqueda para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *param_ranges['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *param_ranges['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *param_ranges['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *param_ranges['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *param_ranges['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *param_ranges['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *param_ranges['reg_lambda']),
    }

    # Definir el espacio de búsqueda para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_ranges['min_frequency'])
    }

    # Crear el modelo con el pipeline

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(min_frequency=encoder_params['min_frequency']), categorical_features)
        ])

    pipeline_xgb = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**xgb_params))
    ])

    start_time = time.time()


    mae = cross_val_score(pipeline_xgb, X_train, y_train, scoring='neg_mean_absolute_error').mean()
    elapsed_time = time.time() - start_time

    # Reportar el tiempo de entrenamiento
    print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

    # Devolver el valor negativo de la métrica para minimizar (optuna maximiza la función objetivo)
    return -mae
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
# Configurar el tiempo límite de la optimización a 5 minutos
study.optimize(objective, n_trials=100, timeout=300)

# Obtener los mejores hiperparámetros y su rendimiento
best_params = study.best_params
best_mae = -study.best_value
num_trials = len(study.trials)

print(f"Número de trials: {num_trials}")
print(f"MAE: {best_mae}")
print(f"Mejores hiperparámetros: {best_params}")

# Guardar el mejor trial en un archivo .pkl
best_trial = study.best_trial
joblib.dump(best_trial, 'mejor_trial_optimizado.pkl')

[I 2023-11-17 01:45:45,639] A new study created in memory with name: no-name-96ac26cc-95e3-4a89-ab5b-c9bc129efa98
[I 2023-11-17 01:45:54,645] Trial 0 finished with value: 9726.548342972827 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 9726.548342972827.


Tiempo de entrenamiento: 8.982993125915527 segundos


[I 2023-11-17 01:45:56,544] Trial 1 finished with value: 5971.39114343281 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 5971.39114343281.


Tiempo de entrenamiento: 1.891035556793213 segundos


[I 2023-11-17 01:45:58,683] Trial 2 finished with value: 8852.264057870689 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_depth': 6, 'max_leaves': 29, 'min_child_weight': 4, 'reg_alpha': 0.13949386065204183, 'reg_lambda': 0.29214464853521815, 'min_frequency': 0.3663618432936917}. Best is trial 1 with value: 5971.39114343281.


...

[I 2023-11-17 01:50:41,831] Trial 47 finished with value: 2364.391142340498 and parameters: {'learning_rate': 0.08132904090979161, 'n_estimators': 731, 'max_depth': 8, 'max_leaves': 94, 'min_child_weight': 3, 'reg_alpha': 0.3947172671749263, 'reg_lambda': 0.27426241569157256, 'min_frequency': 0.046262669620218216}. Best is trial 42 with value: 2278.832142538854.


Tiempo de entrenamiento: 7.7071709632873535 segundos


[I 2023-11-17 01:50:54,633] Trial 48 finished with value: 6625.104216043389 and parameters: {'learning_rate': 0.049810956889591695, 'n_estimators': 886, 'max_depth': 9, 'max_leaves': 74, 'min_child_weight': 2, 'reg_alpha': 0.2475178119116772, 'reg_lambda': 0.17984286957638007, 'min_frequency': 0.1265029822673303}. Best is trial 42 with value: 2278.832142538854.


Tiempo de entrenamiento: 12.745166301727295 segundos
Número de trials: 49
MAE: -2278.832142538854
Mejores hiperparámetros: {'learning_rate': 0.08597817837767156, 'n_estimators': 927, 'max_depth': 8, 'max_leaves': 68, 'min_child_weight': 4, 'reg_alpha': 0.34221694645096484, 'reg_lambda': 0.2012572486356236, 'min_frequency': 0.06664845155954803}


['mejor_trial_optimizado.pkl']

Reportar el número de trials, el MAE y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

**Respuesta:**

En total se realizaron 49 trials, teniendo un mejor MAE de 2278.832142538854, con los siguientes hiperparametros: \

 {'learning_rate': 0.08597817837767156, 'n_estimators': 927, 'max_depth': 8, 'max_leaves': 68, 'min_child_weight': 4, 'reg_alpha': 0.34221694645096484, 'reg_lambda': 0.2012572486356236, 'min_frequency': 0.06664845155954803}
['mejor_trial_optimizado.pkl'] \

Los resultados mejoran respecto a la sección anterior, pasando de un MAE de  2278 a uno de 2327, los resultados exhiben una mejora con respecto a la sección anterior. Este cambio se atribuye al proceso de optimización llevado a cabo por Optuna, que explora el espacio de hiperparámetros proporcionado con el objetivo de encontrar la combinación que minimice el MAE.

Es importante señalar que, si no hubiera existido un límite de tiempo de 5 minutos (timeout), es posible que los resultados hubieran mejorado aún más. No obstante, la mejora observada en este intervalo de tiempo es considerable y, por lo tanto, se considera que los resultados obtenidos son preferibles en términos de eficiencia y calidad.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:


from optuna.integration import XGBoostPruningCallback
def objective(trial):
    # Definir el espacio de búsqueda para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', *param_ranges['learning_rate']),
        'n_estimators': trial.suggest_int('n_estimators', *param_ranges['n_estimators']),
        'max_depth': trial.suggest_int('max_depth', *param_ranges['max_depth']),
        'max_leaves': trial.suggest_int('max_leaves', *param_ranges['max_leaves']),
        'min_child_weight': trial.suggest_int('min_child_weight', *param_ranges['min_child_weight']),
        'reg_alpha': trial.suggest_float('reg_alpha', *param_ranges['reg_alpha']),
        'reg_lambda': trial.suggest_float('reg_lambda', *param_ranges['reg_lambda']),
    }

    # Definir el espacio de búsqueda para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', *encoder_ranges['min_frequency'])
    }

    # Crear el modelo con el pipeline

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(min_frequency=encoder_params['min_frequency']), categorical_features)
        ])

    pipeline_xgb = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**xgb_params))
    ])

    start_time = time.time()
    pruning_callback = XGBoostPruningCallback(trial, 'validation-mae')

    mae = cross_val_score(pipeline_xgb, X_train, y_train, scoring='neg_mean_absolute_error').mean()
    elapsed_time = time.time() - start_time

    # Reportar el tiempo de entrenamiento
    print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

    # Devolver el valor negativo de la métrica para minimizar (optuna maximiza la función objetivo)
    return -mae

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
# Configurar el tiempo límite de la optimización a 5 minutos
study.optimize(objective, n_trials=100, timeout=300, show_progress_bar = True)

# Obtener los mejores hiperparámetros y su rendimiento
best_params = study.best_params
best_mae = -study.best_value
num_trials = len(study.trials)

print(f"Número de trials: {num_trials}")
print(f"MAE: {best_mae}")
print(f"Mejores hiperparámetros: {best_params}")

# Guardar el mejor trial en un archivo .pkl
best_trial = study.best_trial
joblib.dump(best_trial_pruned, 'mejor_trial_optimizado_pruned.pkl')

  0%|          | 0/100 [00:00<?, ?it/s]

Tiempo de entrenamiento: 5.094158411026001 segundos
Tiempo de entrenamiento: 2.101025104522705 segundos
Tiempo de entrenamiento: 7.920328617095947 segundos
Tiempo de entrenamiento: 2.8309621810913086 segundos
Tiempo de entrenamiento: 11.666825294494629 segundos
Tiempo de entrenamiento: 1.4497241973876953 segundos
Tiempo de entrenamiento: 1.7556734085083008 segundos
Tiempo de entrenamiento: 1.0949249267578125 segundos
Tiempo de entrenamiento: 2.079406261444092 segundos
Tiempo de entrenamiento: 12.568557262420654 segundos
Tiempo de entrenamiento: 0.47782158851623535 segundos
Tiempo de entrenamiento: 2.982039451599121 segundos
Tiempo de entrenamiento: 2.140225410461426 segundos
Tiempo de entrenamiento: 5.69849157333374 segundos
Tiempo de entrenamiento: 2.2033820152282715 segundos
Tiempo de entrenamiento: 2.1706128120422363 segundos
Tiempo de entrenamiento: 1.7692129611968994 segundos
Tiempo de entrenamiento: 2.1862826347351074 segundos
Tiempo de entrenamiento: 1.1137683391571045 segundos


['mejor_trial_optimizado.pkl']

Respecto a los resultados obtenidos en esta iteracion de optimizacion con optuna. Se obtiene un MAE sumamente parecido al anterior, esto corresponde a lo esperable de un algoritmo de pruning si es que el experimento anterior llego a tener un buen resultado por chance de no seguir las ramas poco eficientes. En general un algoritmo de pruning permite una optimizacion mucho mas rapida debido a que corta las ramas con malos resultados tempranamente.

De lo anterior se puede decir que el experimento realizado tuvo un buen desempepeño debido a chance y llego a ser casi tan bueno como lo obtenido con pruning. Pero en general se obtienen resultados mucho mas consistentes con pruning.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

In [ ]:
plot_optimization_history(study)

Del Grafico de optimización es posible notar que los mejores resultados se obtienen de forma mucho mas consistente desde el trial numero 12. Anterior a este las mejoras se ajustan bastante abruptamente, pero posterior al trial 12 se obtienen los resultados que son mucho mas cercanos al optimo final.

In [ ]:
plot_parallel_coordinate(study)

De este grafico se observa que las tendencias de entrenamiento se hayan bastante concentradas en hiperparametros como objective_value, learning_rate, min_frequency y reg_alpha. Es decir los optimos se encontraron mucho mas rapido respecto a estos parametros. Por otro lado hiperparametros como profundidad max_depth, max_leaves, y min_child_weight no tuvieron una mayor tendencia en cuanto se tratara de obtener los  mejores valores.

Lo anterior se evidencia por las concentraciones mucho mas altas de lineas y colores mas oscuros en los parametros descrito con mayor tendencia, que en los parametros que no tenian una mayor tendencia, los cuales tenian lineas claras bastante distribuidas entre sus valores posibles.

In [ ]:
plot_param_importances(study)

Lo anterior queda mucho mas claro al analizar este grafico de importancia de hiperparametros, donde es notable el gran impacto que tiene el parametro de min_frequency, el cual esta relacionado con el encoder de las variables categoricas. Por otro lado tambien se puede ver que max_leaves tiene una importancia bastante alta, lo cual no era muy visible en el grafico anterior, esto esta principalmente relacionado a la forma de visualizar la data, en donde parametros con valores enteros son mucho mas dificiles de evaluar que parametros con valores continuos en graficos de coordenadas paralelas.

De todas formas ambos graficos son complementarios en el entendimiento de la optimizacion de estos hiperparametros ya que permiten entender que variables tuvieron mayor variabilidad, mientras que permite visualizar que valores de estas variables obtuvieron los mejores resultados de forma evolutiva.



## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

| Model                     | Mean Absolute Error (MAE) |
|---------------------------|---------------------------|
| DummyRegressor            | 13308.134750658153        |
| XGBRegressor              | 2426.128624135566         |
| XGBRegressor              | 2327.1220129443423        |
| XGBRegressor optimized    | 2278.832142538854         |
| XGBRegressor pruned       | 2278.832142538854         |


De esta tabla se observa que se obtuvieron resultados parecidos en optimized y pruned, pero como descrito anteriormente pruned entrega resultados mucho mas consistentes, por lo cual se escoge este modelo para la predicción final.

In [ ]:
#Predecimos sobre los datos de test
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(min_frequency=best_params['min_frequency']), categorical_features)
    ])


best_param= {'learning_rate': 0.08637102405687036, 'n_estimators': 918, 'max_depth': 7, 'max_leaves': 68, 'min_child_weight': 4, 'reg_alpha': 0.4216902629736669, 'reg_lambda': 0.294200952269262}
pipeline_xgb_pruning= Pipeline([
('date_transform', date_transformer),
('preprocessor', preprocessor),
('regressor', XGBRegressor(**best_params))
])
#Entrenamos el pipeline con los datos de train



pipeline_xgb_pruning.fit(X_train, y_train)


test_predictions_xgb_best_pruning = pipeline_xgb_pruning.predict(X_test)

#Obtenemos el valor del MAE
mae_xgb_best_pruning = mean_absolute_error(y_test, test_predictions_xgb_best_pruning)

#Imprimimos resultados relevantes
print(f'MAE con XGBoost y los mejores hiperparámetros (prunning): {mae_xgb_best_pruning}')
#Guardamos el regresor XGBoost con los mejores hiperparámetros y pruning
joblib.dump(best_trial, 'mejor_trial_optimizado_pruning.pkl')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[02:40:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.




MAE con XGBoost y los mejores hiperparámetros (prunning): 1979.4862108449975


['mejor_trial_optimizado_pruning.pkl']

De lo anterior se observa una clara diferencia entre lo obtenido con las metricas de validación, en donde los resultados mejoran considerablemente llegando a un MAE de 1979. Esto tiene sentido ya que muchas veces los conjuntos de validacion pueden tener sezgos autocontenidos en estos conjuntos por lo cual es posible que al probar el modelo en la data de validación entregue resultados que superaban lo anterior debido a que contien datos mas consistentes. Claro que esto es un tema de chance y depende de las distintas seeds escogidas para el traisplit realizado.


Esto tambien puede estar relacionado al tamaño del conjunto de datos o un ajuste a los datos de validación que no resultos ser bueno en esta data simplemente por la variabilidad intrinseca de los conjuntos de datos obtenidos al realizar una separacion de entrenamiento.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>